In [ ]:
##Python venv: search_agent_poc

In [1]:
##This module will take the serialized dictionary got out of PDF Parsing ann try to extract
##Semantic Knowldge like identifying 
## 1.Important Objects/Entities
## 2.Deduplicate Entities
## 3.Extracting Relations
## 4.Extract the main Ideas/Topics around Each Page
## 5.Link the different topics via diffrent entities/Objects
## 6.Break down the document by pages instead of Chunks .
## 7.If a page does not fit a chunk then chunk them extract information and then deduplicate the information across
## the page.

#Next Steps:
## 5.Try to Seggregate the BigPDF on Sections.
## 8.Try To Find Common Objects or ideas that link these sections.
## 9.Try 

In [14]:
import pickle
import pandas as pd
#rag_test_input_path='/home/matrix4284/MY_GEN_AI_PROJECTS/RAG/GraphRAG/graphrag-local-ollama/ragtest/input/'+file_name
import os
# importing shutil module
import shutil

from langchain_community.chat_models import ChatOllama

from langchain_community.embeddings import HuggingFaceEmbeddings

from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

from sentence_transformers import SentenceTransformer
#embeddings = SentenceTransformer("all-MiniLM-L6-v2")

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

import json
import operator
from typing import Annotated, Sequence, TypedDict

from langchain import hub
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_core.messages import BaseMessage, FunctionMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnablePassthrough
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
import json
import operator
from typing import Annotated, Sequence, TypedDict

from langchain import hub
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_core.messages import BaseMessage, FunctionMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnablePassthrough
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import XMLOutputParser

In [15]:
model_name="llama3.1"
book_text=''
page_text=''
file_name='Kubernetes_in_action_text_only'
extension='.txt'
start_page_idx=99
end_page_index=480
#full_filename=file_name+'_'+str(page_idx)+extension
#full_filename
pdf_enrichment_output_dir='./pdf_enriched_output/'

In [16]:
#LLM Model for Prompt Tuning
llm = ChatOllama(base_url="http://192.168.50.100:11434",model=model_name)

#embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
model_kwargs = {'device': 'cpu'}

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2",model_kwargs = model_kwargs)

##Define Vectorstore
vectorstore = Chroma(embedding_function=embeddings, persist_directory="./chroma_kubernetes_in_action")

In [17]:
# Load data (deserialized data)
with open(pdf_enrichment_output_dir+'pdf_enriched_content_dict_stage2_phase1.pickle', 'rb') as handle:
    document_dict_deserialized = pickle.load(handle)

In [18]:
len(document_dict_deserialized)

479

In [19]:
document_dict_deserialized_stage2=document_dict_deserialized

In [20]:
len(document_dict_deserialized_stage2)

479

In [9]:
##Definitions of Individual Enrichment Modules######

def extract_summary_per_page(page_text):
    
    
    parser = XMLOutputParser()

# Prompt
    prompt = PromptTemplate(
            template="""You are a computer technology expert who has mastery in Kubernetes,Docker \n
            Machine Larning,Generative AI,Natural Language Understanding and Computer Vision.\n
            You have to deeply study a document page and extract a small summary within 150 words that will\n
            be enough to represent all information for that page.
            There is no need to mention any header statement before the summary.
            Wrap the summary in a json with key named summary.
            Output the json and nothing else no headers no footers.
            Here is the document page: \n\n {context} \n\n""",
            input_variables=["context"],
            ###Introduced by Kaustav while experimenting with XMLParsers
            #partial_variables={"format_instructions": parser.get_format_instructions()},
    )

    # Chain
    #Commented out By Kaustav 20/08/2024 as str output parser is giving some unwanted text #####
    chain = prompt | llm | StrOutputParser()
   
    




    #Trying out XML Output parser by Kautsva added 20/08/2024####
    #chain = prompt | llm | parser
    
    

    # Score
    #filtered_docs = []

    output = chain.invoke(
        {
                "context": page_text,
        }
    )
    
    json_output = output
    json_output=json_output.split('{')[1]
    #page_output_json=json.loads(output)
    
    
    #print('JSON OUTPUT:')
    #print(json_output)
    
    json_output='{'+json_output
    
    json_output=json_output.split('}')[0]
    
    json_output=json_output+'}'
    #page_output_json=json.loads(output)
    #return page_output_json
    return json_output
    
    #return page_output_json
    #return output

    
##Definitions of Individual Enrichment Modules######

def extract_highlights_per_page(page_text):
    
    
    parser = XMLOutputParser()

# Prompt
    prompt = PromptTemplate(
            template="""You are a computer technology expert who has mastery in Kubernetes,Docker,\n
            Machine Learning,Generative AI,Natural Language Understanding and Computer Vision.\n
            You have to deeply study a document page and extract upto maximum of 10 highlights from the page \n
            content.\n
            Extract the highlights following the mentioned rules below:\n
            1.For each highlight wrap it up in json with the key named highlight.\n
            2.After all the highlights have been extracted collate them into a list of json.\n
            Out should only contain the list of json and no other words or character or sentences.\n
            Here is the document page: \n\n {context} \n\n""",
            input_variables=["context"],
            ###Introduced by Kaustav while experimenting with XMLParsers
            #partial_variables={"format_instructions": parser.get_format_instructions()},
    )

    # Chain
    #Commented out By Kaustav 20/08/2024 as str output parser is giving some unwanted text #####
    chain = prompt | llm | StrOutputParser()
   
    




    #Trying out XML Output parser by Kautsva added 20/08/2024####
    #chain = prompt | llm | parser
    
    

    # Score
    #filtered_docs = []

    output = chain.invoke(
        {
                "context": page_text,
        }
    )
    
    
    json_output=output.split('[')[1]
    #page_output_json=json.loads(output)
    
    
    #print('JSON OUTPUT:')
    #print(json_output)
    
    json_output='['+json_output
    
    
    json_output=reverse(json_output)
    
    print('Reversed JSON OUTPUT:')
    print(json_output)
    
    json_output=json_output.split(']')[1]
    
    json_output=reverse(json_output)
    #json_output=json_output.rsplit(']')[-1]
    #page_output_json=json.loads(output)
    
    
    #print('JSON OUTPUT:')
    #print(json_output)
    
    json_output= json_output + ']'
    
    print('JSON OUTPUT:')
    print(json_output)
    
    
    #page_output_json=json.loads(output)
    #return page_output_json
    return json_output
    
    #return page_output_json
    #return output

def entity_collector_per_page(entity_lst):

    entities=[]
    
    for entity in entity_lst:
        print("Entity:")
        print(entity)
        entity_name=entity['entity']
        entities.append(entity_name)
    return list(set(entities))


def enrich_page(page_idx):
    
    #print("Page Number")
    #print(page_idx)
    
    page_text=document_dict_deserialized_stage2[page_idx]['text']
    
    ##Entity Extraction Enrichment
    page_summary_txt=extract_summary_per_page(page_text)
    
    #print("Page Summary Text")
    print(page_summary_txt)
    
    #print("Page Summary Json")
    #print(page_entity_lst_dict)
    
    #page_highlights_lst_dict_json={}
    #page_highlights_lst_dict_json['summary']=page_highlights_lst_dict
    
    #page_highlights_lst_dict_json=json.loads(page_highlights_lst_dict)
    document_dict_deserialized_stage2[page_idx]['summary']=json.loads(page_summary_txt.strip())["summary"]
    

In [10]:
for idx in range(start_page_idx,end_page_index):
   
        enrich_page(idx)
        print("Done for page number:"+str(idx))

{"summary": "Headless services allow clients to discover and connect directly to pods using DNS names. They provide load balancing through DNS round-robin, but unlike regular services, they don't use a service proxy. To include unready pods in a service's DNS lookup, add the 'service.alpha.kubernetes.io/tolerate-unready-endpoints: true' annotation. When troubleshooting services, check if the pods are running and the service is properly configured, as issues can arise from incorrect labels or annotations."}
Done for page number:99
{"summary": "To troubleshoot Kubernetes service-related problems, ensure you're connecting to the cluster IP from within the cluster. Verify that readiness probes are succeeding and check the Endpoints object with kubectl get endpoints. Ensure you're accessing the correct port and try connecting directly to the pod IP. If still issues, check if your app is only binding to localhost."}
Done for page number:100
{"summary": "Services enable clients to discover an

{"summary": "Kubernetes PersistentVolumes are managed by cluster administrators who set up and register underlying storage. Users create a PersistentVolumeClaim manifest, specifying size and access mode requirements. Kubernetes finds an available PersistentVolume and binds it to the claim. The volume can be used within a pod, but other users cannot use the same PersistentVolume until it is released."}
Done for page number:120
{"summary": "To attach disk storage to containers, create a PersistentVolume (PV) that specifies its capacity, read/write permissions, and storage type. Create the PV with kubectl create command. Afterward, list all PVs using kubectl get pv to check if it's available. A PV is cluster-level resource like nodes and doesn't belong to any namespace."}
Done for page number:121
{"summary": "To use persistent storage in a Kubernetes pod, you must claim a PersistentVolume by creating a PersistentVolumeClaim. This is a separate process from creating a pod to ensure the cla

{"summary": "ConfigMaps and Secrets in Kubernetes allow passing configuration options to containers through command-line arguments or environment variables, where values are passed as arrays or lists of key-value pairs, with options for specifying custom variables for each container within a pod, but not at the pod level."}
Done for page number:139
{"summary": "This section explains how to make environment variables configurable within a container, and how to specify these variables when defining a container in Kubernetes. It provides examples of modifying a bash script to use an environment variable, and defining a pod with an environment variable passed to the container."}
Done for page number:140
{"summary": "ConfigMaps and Secrets allow decoupling application configuration from the pod descriptor, enabling reusability across environments. Environment variables can reference previously defined variables using $(VAR) syntax. Hardcoding environment variables leads to separate pod defi

{"summary": "You'll create a Secret called fortune-https to configure Nginx container to serve HTTPS traffic by generating a certificate, private key, and dummy file. The files are added to the Secret using kubectl create secret generic command, similar to creating ConfigMaps. This creates a volume in each pod with a default-token Secret."}
Done for page number:159
{"summary": "Secrets in Kubernetes can contain sensitive or binary data and are encoded in Base64 for storage in plain-text formats like YAML or JSON. Unlike ConfigMaps, which display clear text, Secrets entries show as encoded strings. This allows for storing binary values within YAML or JSON manifests."}
Done for page number:160
{"summary": "Kubernetes allows setting a Secret's values through the stringData field for sensitive non-binary data. The field is write-only and values are displayed under 'data' in YAML format. When exposed to a container or environment variable, values are decoded and written in their actual form

{"summary": "This chapter focuses on accessing pod metadata and other resources from applications. The Kubernetes REST API server can be accessed using curl, allowing applications to retrieve resources such as Jobs. To access a specific Job instance, the name and namespace must be specified in the URL. From within a pod, one must find the location of the API server, verify its authenticity, and authenticate with the server before making requests."}
Done for page number:181
{"summary": "To talk to the Kubernetes API server, create a pod using the tutum/curl image and run a bash shell inside its container. Then, use curl to access the API server from within that shell. To find the API server's address, look up the KUBERNETES_SERVICE_HOST and KUBERNETES_SERVICE_PORT variables, which are automatically configured for each service."}
Done for page number:182
{"summary": "To access pod metadata and resources, applications can use the environment variables provided by Kubernetes. However, it's

...

{"summary": "This chapter covers best practices for developing apps on Kubernetes, including understanding which resources appear in a typical application, adding lifecycle hooks, properly terminating an app without breaking client requests, making apps easy to manage, using init containers, and developing locally with Minikube."}
Done for page number:420
{"summary": "A typical application manifest contains one or more Deployment and/or StatefulSet objects, including pod templates with containers, liveness and readiness probes, and services exposed through LoadBalancer, NodePort, or Ingress resources. The pod templates reference Secrets for container image pulling and internal process usage, which are usually assigned to ServiceAccounts and created by the operations team."}
Done for page number:421
{"summary": "Pods are the most important Kubernetes resource, running one application each. They can be killed and relocated by Kubernetes due to scale-down requests or node relocations. App

{"summary": "It's recommended to use tags with version designators instead of 'latest', and label resources with multiple dimensions for easier management. Additionally, describe each resource through annotations, providing contact information and metadata such as build and version details. This helps in understanding why a container terminated or is terminating continuously."}
Done for page number:441
{"summary": "In Kubernetes, you can make it easy for ops people to triage issues by including a termination message in your log files. This can be achieved by writing a message to a specific file in the container's filesystem before exiting. The default path is /dev/termination-log, but you can override this by setting the terminationMessagePath field in the container definition. When a pod crashes, kubectl describe will show the reason for the crash without needing to inspect logs."}
Done for page number:442
{"summary": "Kubernetes app development best practices include logging to stand

{"summary": "A custom API object is defined with a controller that creates a Service and Deployment upon receiving an ADDED event. The API server responds with a 201 Created response, creating resources such as Deployments, Services, and Pods. Users can deploy static websites in seconds without knowing Kubernetes resources, except the custom Website resource. However, users can create invalid objects due to lack of validation schema, but the controller can validate objects and update error messages."}
Done for page number:460
{"summary": "Kubernetes allows custom object validation by enabling the CustomResourceValidation feature gate and specifying a JSON schema in the CRD, but a better approach is to implement a custom API server for custom objects through API server aggregation, which can validate objects directly and provide a single point of access for clients."}
Done for page number:461
{"summary": "To extend Kubernetes, a custom API server can be added by deploying it as a pod an

{"summary": "To use kubectl with multiple clusters or namespaces, you can configure the location of the kubeconfig file using the KUBECONFIG environment variable. This allows you to specify multiple config files, which are then used by kubectl. The kubeconfig file itself has four sections: a list of clusters, users, contexts, and the current context. A context defines a kubectl context, while a user's credentials are stored in a separate section."}
Done for page number:478


IndexError: list index out of range

In [13]:
###The main contents of Kubernets In Action book is contained from page 87 to 565####
####Thats why we will only take the sliced portion i.e. from page 87 to 565 of the pdf dictionary list#####
import pickle

# Store data (serialize in a pickle) upto page 102
with open('./pdf_enriched_output/pdf_enriched_content_dict_stage2_phase1.pickle', 'wb') as handle:
    pickle.dump(document_dict_deserialized_stage2, handle, protocol=pickle.HIGHEST_PROTOCOL)